In [74]:
import joblib
import pandas as pd
import numpy as np

players = pd.read_csv("../data/players.csv")

teams = pd.read_csv("../data/teams.csv")

info = pd.read_csv("../data/info.csv")

team_divisions = pd.read_csv('../data/team_divisions.csv')


model_ready_data = joblib.load("../data/pkls/model_ready_data_dict.pkl")
X = model_ready_data['X']
players_wide  = model_ready_data['players_wide']

y = model_ready_data['y']
log_ids = joblib.load("../data/pkls/log_id/log_id.pkl")

In [1]:
import pandas as pd
import numpy as np

def high_corr_pairs(corr_matrix, threshold=0.8, absolute=True):
    corr_matrix = corr_matrix.copy()
    # Mask upper triangle and diagonal
    mask = np.tril(np.ones(corr_matrix.shape), k=-1).astype(bool)
    corr_unstacked = corr_matrix.where(mask).stack()
    
    # Filter by threshold
    if absolute:
        corr_filtered = corr_unstacked[abs(corr_unstacked) >= threshold]
    else:
        corr_filtered = corr_unstacked[corr_unstacked >= threshold]
    
    # Format as dataframe
    return corr_filtered.reset_index().rename(
        columns={'level_0': 'var1', 'level_1': 'var2', 0: 'correlation'}
    )

corrs = high_corr_pairs(X.corr(), threshold=0.85)


NameError: name 'X' is not defined

In [ ]:
team_divisions = team_divisions.merge(teams[['id','team','winner']],on = ['id','team'])
X_pre_bin = X.copy()
X_pre_bin['winner'] = y

X_pre_bin  = X_pre_bin.merge(team_divisions[['id','winner','division_weight']],on = ['id','winner'])
weights = X_pre_bin['division_weight'].copy()
X_pre_bin = X_pre_bin.drop(['id','winner','division_weight'],axis=1)

In [77]:
weights = X_pre_bin['division_weight'].copy()
X_pre_bin = X_pre_bin.drop(['id','winner','division_weight'],axis=1)

In [ ]:
from optbinning import OptimalBinning
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
problem =[]
X_binned = pd.DataFrame()
for col in X_pre_bin.columns:
    try:    
        optb = OptimalBinning(
            max_n_bins=5,
            name = col,
            min_bin_size = .05,
            
        )
        optb.fit(X[col],y,sample_weight=weights)
        binned_col = optb.transform(x = X[col],metric = "bins")
        X_binned = X_binned.copy()
        X_binned[col] = binned_col

    except Exception as e:
        problem.append(col)

In [79]:
X_exploded = pd.get_dummies(X_binned).copy()
X_exploded['win'] = y.copy()
X_exploded['id'] = X['id'].copy()
X_exploded = X_exploded.merge(info[['id','clean_map_name']])

In [80]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from pandas.errors import PerformanceWarning
warnings.filterwarnings("ignore", category=PerformanceWarning)

def conditional_target_encoding_vectorized(df, feature_cols, map_col='clean_map_name', target_col='win', id_col='id', n_splits=5, smoothing=5):
    """
    Fast k-fold leakage-free map-adjusted target encoding for binary feature columns (vectorized version).
    Preserves ID, map, and target columns for easy merging back.
    """
    # Initialize output with identifier columns
    df_encoded = df[[id_col, map_col, target_col]].copy()
    
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    # Prepare empty columns for map-adjusted features
    for feature in feature_cols:
        df_encoded[feature + '_map_adj'] = 0.0
    
    for train_idx, val_idx in kf.split(df):
        train, val = df.iloc[train_idx], df.iloc[val_idx]
        
        # Compute global feature-bin win rates for smoothing
        global_rates = train[feature_cols].multiply(train[target_col], axis=0).sum() / train[feature_cols].sum()
        
        for feature in feature_cols:
            # Only rows where feature is active
            train_feat = train[train[feature] == 1].copy()
            
            # Group by map and compute mean and count
            map_feat_stats = train_feat.groupby(map_col)[target_col].agg(['mean', 'count']).reset_index()
            map_feat_stats['smoothed'] = (
                (map_feat_stats['count'] * map_feat_stats['mean'] + smoothing * global_rates[feature]) / 
                (map_feat_stats['count'] + smoothing)
            )
            
            # Merge smoothed map-feature encoding with validation fold
            val_feat = val[[feature, map_col]].copy()
            val_feat = val_feat.merge(map_feat_stats[[map_col, 'smoothed']], on=map_col, how='left')
            
            # Fill missing maps with global rate
            val_feat['smoothed'] = val_feat['smoothed'].fillna(global_rates[feature])
            
            # Apply encoding only to active feature rows
            df_encoded.loc[val_idx, feature + '_map_adj'] = val_feat['smoothed'] * val_feat[feature]
    
    return df_encoded

# Example usage:
feature_cols = [col for col in X_exploded.columns if col not in ['id', 'clean_map_name', 'win']]
df_map_adjusted = conditional_target_encoding_vectorized(X_exploded, feature_cols, map_col='clean_map_name', target_col='win', id_col='id', n_splits=5)


In [81]:
from pandas.errors import PerformanceWarning
warnings.filterwarnings("ignore", category=PerformanceWarning)
final_df = pd.DataFrame()
for col in X_binned.columns:
        
    binned_cols = [bins for bins in df_map_adjusted.columns if bins.startswith(col)]
    collected_column = df_map_adjusted[binned_cols].T.sum()
    final_df[col + "_map_adj"] = collected_column.copy()

In [83]:
final_df

,demoman_dapd_map_adj,scout_pocket_dapd_map_adj,scout_roamer_dapd_map_adj,soldier_pocket_dapd_map_adj,soldier_roamer_dapd_map_adj,demoman_dapm_map_adj,scout_pocket_dapm_map_adj,scout_roamer_dapm_map_adj,soldier_pocket_dapm_map_adj,soldier_roamer_dapm_map_adj,scout_pocket_offclass_pct_map_adj,scout_roamer_offclass_pct_map_adj,demoman_kill_pct_map_adj,scout_pocket_kill_pct_map_adj,scout_roamer_kill_pct_map_adj,soldier_pocket_kill_pct_map_adj,scout_pocket_deaths_pct_map_adj,scout_roamer_deaths_pct_map_adj,soldier_pocket_deaths_pct_map_adj,soldier_roamer_deaths_pct_map_adj,demoman_cpc_pct_map_adj,scout_pocket_cpc_pct_map_adj,scout_roamer_cpc_pct_map_adj,soldier_pocket_cpc_pct_map_adj,soldier_roamer_cpc_pct_map_adj,demoman_assists_pct_map_adj,scout_pocket_assists_pct_map_adj,soldier_pocket_assists_pct_map_adj,scout_pocket_dt_pct_map_adj,scout_roamer_dt_pct_map_adj,soldier_pocket_dt_pct_map_adj,soldier_roamer_dt_pct_map_adj,demoman_dt_real_pct_map_adj,scout_pocket_dt_real_pct_map_adj,soldier_pocket_dt_real_pct_map_adj,soldier_roamer_dt_real_pct_map_adj,scout_roamer_hr_pct_map_adj,soldier_pocket_hr_pct_map_adj,soldier_roamer_hr_pct_map_adj,demoman_hroi_real_map_adj,...,soldier_roamer_spy_deaths_class_kdapd_map_adj,medic_dapd_map_adj,medic_dapm_map_adj,medic_kill_pct_map_adj,medic_deaths_pct_map_adj,medic_cpc_pct_map_adj,medic_dt_pct_map_adj,medic_dt_real_pct_map_adj,medic_assistspd_map_adj,medic_demoman_deaths_class_kdapd_map_adj,medic_engineer_deaths_class_kdapd_map_adj,medic_heavyweapons_deaths_class_kdapd_map_adj,medic_scout_deaths_class_kdapd_map_adj,medic_sniper_deaths_class_kdapd_map_adj,medic_soldier_deaths_class_kdapd_map_adj,medic_demoman_assists_class_kdapd_map_adj,medic_medic_assists_class_kdapd_map_adj,medic_scout_assists_class_kdapd_map_adj,medic_sniper_assists_class_kdapd_map_adj,medic_soldier_assists_class_kdapd_map_adj,medic_spy_assists_class_kdapd_map_adj,medic_healpm_map_adj,medic_dtpm_map_adj,medic_dt_realpm_map_adj,medic_medkits_hppm_map_adj,medic_hrpm_map_adj,medic_deathspm_map_adj,medic_heavyweapons_assists_class_kdapd_map_adj,medic_pyro_deaths_class_kdapd_map_adj,medic_spy_deaths_class_kdapd_map_adj,medic_engineer_assists_class_kdapd_map_adj,medic_pyro_assists_class_kdapd_map_adj,medic_avg_time_before_healing_map_adj,medic_avg_time_to_build_map_adj,medic_avg_uber_length_map_adj,medic_biggest_advantage_lost_map_adj,medic_deaths_with_95_99_uber_rate_map_adj,medic_deaths_within_20s_after_uber_rate_map_adj,medic_drops_rate_map_adj,medic_uberspm_map_adj
0,0.533466,0.581482,0.446725,0.754930,0.719723,0.477755,0.417435,0.417590,0.322109,0.355262,0.355724,0.394839,0.518266,0.491085,0.512398,0.485776,0.485762,0.461575,0.532932,0.397745,0.536658,0.514338,0.563130,0.538197,0.581356,0.534360,0.487787,0.527139,0.505742,0.468409,0.461878,0.502402,0.453502,0.502908,0.454274,0.443980,0.511796,0.513075,0.483064,0.611493,...,0.497482,0.524408,0.511628,0.545186,0.441379,0.496180,0.388834,0.418129,0.582906,0.420184,0.497482,0.497482,0.407452,0.524375,0.483524,0.485320,0.664379,0.417240,0.414223,0.357687,0.497502,0.359756,0.655617,0.527218,0.541240,0.463638,0.509223,0.459418,0.497482,0.502660,0.497482,0.484857,0.693631,0.425502,0.501799,0.503442,0.514577,0.594906,0.543199,0.538016
1,0.166017,0.392110,0.208505,0.546379,0.391000,0.351762,0.538158,0.418503,0.687068,0.547402,0.627057,0.485881,0.522903,0.541950,0.517739,0.530997,0.452429,0.455447,0.437924,0.407483,0.543318,0.183185,0.573169,0.545396,0.589058,0.511677,0.539409,0.532125,0.603894,0.562146,0.461448,0.532983,0.455051,0.475476,0.459396,0.491395,0.518203,0.497118,0.490194,0.613688,...,0.503637,0.703567,0.452173,0.509636,0.369783,0.632711,0.395208,0.424113,0.323680,0.431600,0.503637,0.503637,0.456879,0.331182,0.474535,0.378797,0.391120,0.291838,0.425180,0.270670,0.471762,0.368288,0.238162,0.129639,0.618342,0.474154,0.166927,0.843285,0.503637,0.452687,0.503637,0.489387,0.578130,0.425828,0.444304,0.502820,0.518098,0.607834,0.464287,0.123561
2,0.718185,0.922221,0.630415,0.757380,0